In [ ]:
import pandas as pd
import re
import sys
import os
import re

In [ ]:
data_oranda= pd.read_csv("../data/oranda_review.csv")
data_oranda.head()

,이름,작성일자,평점,리뷰 내용,만족도
0,솔직한깡지,2024.11.16,5,지난번 아이 생일때 선생님들 드리려고 샀다가몇개 먹어보고 맛있어서 재구매했어요 장바...,맛있어요
1,김오성,2024.11.11,5,우주오란다 플래인 강정 주문햇습니다 이거 반틈은 내가 먹고 반틈은 울아부지 드릴껍니...,맛있어요
2,김오성,2024.09.26,5,ㅈㅣ는 84세이신 울아부지랑 둘이서 살고 있습니다 울아부지 옛날에는 장승같이 커다란...,맛있어요
3,이*진,2024.09.22,4,《 우주오란다 》 * 용량 : 30g × 12개 * 소비기한 : 2025.04...,맛있어요
4,36참나리,2024.09.22,5,"클래식한 간식 중 최고봉, 오란다.달달한 간식이 생각나 주문했다.옛날 문방구에서 사...",맛있어요


In [ ]:
data_oranda.shape

(1500, 5)

# 데이터 필터링

등록된 리뷰내용이 있는 것만 가져오기

In [ ]:

filtered_data_oranda = data_oranda[data_oranda["리뷰 내용"] != "등록된 리뷰내용이 없습니다"]
filtered_data_oranda


,이름,작성일자,평점,리뷰 내용,만족도
0,솔직한깡지,2024.11.16,5,지난번 아이 생일때 선생님들 드리려고 샀다가몇개 먹어보고 맛있어서 재구매했어요 장바...,맛있어요
1,김오성,2024.11.11,5,우주오란다 플래인 강정 주문햇습니다 이거 반틈은 내가 먹고 반틈은 울아부지 드릴껍니...,맛있어요
2,김오성,2024.09.26,5,ㅈㅣ는 84세이신 울아부지랑 둘이서 살고 있습니다 울아부지 옛날에는 장승같이 커다란...,맛있어요
3,이*진,2024.09.22,4,《 우주오란다 》 * 용량 : 30g × 12개 * 소비기한 : 2025.04...,맛있어요
4,36참나리,2024.09.22,5,"클래식한 간식 중 최고봉, 오란다.달달한 간식이 생각나 주문했다.옛날 문방구에서 사...",맛있어요
...,...,...,...,...,...
837,정*운,2024.07.27,5,배송속도도 제품품질도 맛도 좋아요,맛있어요
840,박아름,2023.12.01,5,빠른배송에 너무 맛있어요!,예상보다 맛있어요
841,변*곤,2024.05.27,5,배송도 빠르고 파손없이 잘 수령하였습니다.,맛있어요
842,김*화,2023.12.07,5,배송도 빠르고~너무 맛잇어요~~~,예상보다 맛있어요


In [ ]:
df = filtered_data_oranda

불용어 제거 & 특수문자 제거

In [ ]:
from konlpy.tag import Kkma #형태소 분석기
from konlpy.tag import Okt

kkma = Kkma()
okt = Okt()

In [ ]:
# sys.path.append(os.path.dirname('PyKoSpacing/'))
# from pykospacing import spacing #한글 문장에서 띄어쓰기 보정 
# spacing = Spacing()  # Spacing 객체 초기화
# pip install git+https://github.com/haven-jeon/PyKoSpacing.git

def preprocessing(review, filter=None):
    total_review = ''
    #인풋리뷰
    for idx in range(len(review)):
        r = review[idx]
        #하나의 리뷰에서 문장 단위로 자르기
        #불필요한 정보제거 name의 각 단어를 review에서 제거 
        # 이름이 주어졌을 경우에만 이름 정보 제거
        if filter:
            filter_parts = filter.split(' ')
            for part in filter_parts:
                r = re.sub(part, '', r)
        sentence = r
        #특수문자, 영어 알파벳, 초성/중성/종성(예: "ㄱ", "ㅏ" 등)을 제거.
        sentence = re.sub('\n','',sentence)
        sentence = re.sub('\u200b','',sentence)
        sentence = re.sub('\xa0','',sentence)
        sentence = re.sub('([a-zA-Z])','',sentence)
        sentence = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',sentence)
        sentence = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',sentence)
        #전처리 후 문장이 비어있으면 다음 리뷰로 넘어감 
        if len(sentence) == 0:
            continue
        #동사를 기본형으로 변환
        sentence = okt.pos(sentence, stem = True)
        word = []
        #명사가 아닌 경우 건너띔(list에 추가 안함함)
        for i in sentence:
            if not i[1] == 'Noun':
                continue
            if len(i[0]) == 1:
                continue
            word.append(i[0])
        #단어들을 공백으로 연결하여 하나의 문장으로 변환
        #문장 끝에 마침표 추가
        #전체 리뷰에 추가
        word = ' '.join(word)
        word += '. '
        total_review += word
    return total_review

In [ ]:
# 전처리 수행 #21초 정도 걸리는듯듯
df['전처리된_리뷰'] = df.apply(lambda row: preprocessing([row['리뷰 내용']]), axis=1)

C:\Users\zin\AppData\Local\Temp\ipykernel_13732\447569505.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['전처리된_리뷰'] = df.apply(lambda row: preprocessing([row['리뷰 내용']]), axis=1)


In [ ]:
df.head()

,이름,작성일자,평점,리뷰 내용,만족도,전처리된_리뷰
0,솔직한깡지,2024.11.16,5,지난번 아이 생일때 선생님들 드리려고 샀다가몇개 먹어보고 맛있어서 재구매했어요 장바...,맛있어요,지난번 아이 생일 선생님 몇개 보고 장바구니 종종 할인 행사 할인 전혀 장바구니 할...
1,김오성,2024.11.11,5,우주오란다 플래인 강정 주문햇습니다 이거 반틈은 내가 먹고 반틈은 울아부지 드릴껍니...,맛있어요,우주 래인 강정 햇습니 아부지 드릴 아부지 우주 래인 강정 옛날 이제 나이 무가 요...
2,김오성,2024.09.26,5,ㅈㅣ는 84세이신 울아부지랑 둘이서 살고 있습니다 울아부지 옛날에는 장승같이 커다란...,맛있어요,아부지 둘이서 아부지 옛날 장승 모습 는데 나이 가부장 잘못 뚜디리 기억 아부지 아...
3,이*진,2024.09.22,4,《 우주오란다 》 * 용량 : 30g × 12개 * 소비기한 : 2025.04...,맛있어요,우주 용량 소비 기한 구매 이유 물엿 코팅 달달 식감 과자 특유 냄새 추억 우주 쿠...
4,36참나리,2024.09.22,5,"클래식한 간식 중 최고봉, 오란다.달달한 간식이 생각나 주문했다.옛날 문방구에서 사...",맛있어요,클래식 간식 최고봉 달달 간식 주문 옛날 문방구 오란 입천장 이번 구매 수제 강정 ...


특정 단어 제거(이벤트, 체험단 등)

In [ ]:
# def remove_rows_with_keywords(dataframe, column_name, keywords):
#     """
#     특정 단어가 포함된 문장을 제거하는 함수.

#     Args:
#         dataframe (pd.DataFrame): 데이터프레임
#         column_name (str): 처리할 열 이름
#         keywords (list): 제거할 단어 리스트

#     Returns:
#         pd.DataFrame: 특정 단어가 포함된 행이 제거된 데이터프레임
#     """
#     # 특정 단어가 포함된 행 제거
#     pattern = "|".join(keywords)  # 키워드 리스트를 정규식 패턴으로 변환
#     filtered_df = dataframe[~dataframe[column_name].str.contains(pattern, na=False)]
#     return filtered_df

# # 사용 예시
# keywords = ["이벤트", "체험단"]
# filtered_data_oranda = remove_rows_with_keywords(data_oranda, "리뷰 내용", keywords)

# # 결과 확인
# print(filtered_data_oranda)
# len(filtered_data_oranda)

# soynlp

In [1]:
import pandas as pd
import re
from soynlp.noun import LRNounExtractor_v2
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline


/opt/homebrew/Caskroom/miniconda/base/envs/coreview/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
file_path = '../data/oranda_review.csv'
df = pd.read_csv(file_path)
df.head()

,이름,작성일자,평점,리뷰 내용,만족도
0,솔직한깡지,2024.11.16,5,지난번 아이 생일때 선생님들 드리려고 샀다가몇개 먹어보고 맛있어서 재구매했어요 장바...,맛있어요
1,김오성,2024.11.11,5,우주오란다 플래인 강정 주문햇습니다 이거 반틈은 내가 먹고 반틈은 울아부지 드릴껍니...,맛있어요
2,김오성,2024.09.26,5,ㅈㅣ는 84세이신 울아부지랑 둘이서 살고 있습니다 울아부지 옛날에는 장승같이 커다란...,맛있어요
3,이*진,2024.09.22,4,《 우주오란다 》 * 용량 : 30g × 12개 * 소비기한 : 2025.04...,맛있어요
4,36참나리,2024.09.22,5,"클래식한 간식 중 최고봉, 오란다.달달한 간식이 생각나 주문했다.옛날 문방구에서 사...",맛있어요


In [9]:
def preprocess_text(text):
    # 특수문자, 숫자, 불필요한 공백 제거
    text = re.sub(r'\s+', ' ', text)  # 다중 공백 -> 단일 공백
    text = re.sub(r'[^\w\s]', '', text)  # 특수문자 제거
    text = re.sub(r'\d+', '', text)  # 숫자 제거
    return text.strip()

df = df[~df['리뷰 내용'].str.contains("등록된 리뷰내용이 없습니다", na=False)]
print(f"남은 리뷰 개수: {len(df)}")

남은 리뷰 개수: 810


In [10]:
df['preprocessed_review'] = df['리뷰 내용'].apply(preprocess_text)
df.head()

,이름,작성일자,평점,리뷰 내용,만족도,preprocessed_review
0,솔직한깡지,2024.11.16,5,지난번 아이 생일때 선생님들 드리려고 샀다가몇개 먹어보고 맛있어서 재구매했어요 장바...,맛있어요,지난번 아이 생일때 선생님들 드리려고 샀다가몇개 먹어보고 맛있어서 재구매했어요 장바...
1,김오성,2024.11.11,5,우주오란다 플래인 강정 주문햇습니다 이거 반틈은 내가 먹고 반틈은 울아부지 드릴껍니...,맛있어요,우주오란다 플래인 강정 주문햇습니다 이거 반틈은 내가 먹고 반틈은 울아부지 드릴껍니...
2,김오성,2024.09.26,5,ㅈㅣ는 84세이신 울아부지랑 둘이서 살고 있습니다 울아부지 옛날에는 장승같이 커다란...,맛있어요,ㅈㅣ는 세이신 울아부지랑 둘이서 살고 있습니다 울아부지 옛날에는 장승같이 커다란 모...
3,이*진,2024.09.22,4,《 우주오란다 》 * 용량 : 30g × 12개 * 소비기한 : 2025.04...,맛있어요,우주오란다 용량 g 개 소비기한 까지 구매 이유 어릴 때 맛나게 먹던...
4,36참나리,2024.09.22,5,"클래식한 간식 중 최고봉, 오란다.달달한 간식이 생각나 주문했다.옛날 문방구에서 사...",맛있어요,클래식한 간식 중 최고봉 오란다달달한 간식이 생각나 주문했다옛날 문방구에서 사 먹던...


In [11]:
def extract_important_words(reviews, top_k=10):
    noun_extractor = LRNounExtractor_v2(verbose=False)
    combined_text = " ".join(reviews)
    nouns = noun_extractor.train_extract([combined_text])
    sorted_nouns = sorted(nouns.items(), key=lambda x: x[1].frequency, reverse=True)
    return [word for word, score in sorted_nouns[:top_k]]

important_words = extract_important_words(df['preprocessed_review'])


print("Important Words:", important_words)


Important Words: ['맛', '오란다', '딱딱', '말랑말랑', '좋아', '잘', '구매', '맛있어요', '또', '주문']


### WordExtractor

In [25]:
from soynlp.word import WordExtractor
import pandas as pd

def df_sumWordSentence_soynlp_WE(df, review_col, rate_col=None, stopwords=None, top_k=5, full_merge=False, rate_merge=False):
    """
    데이터프레임에서 특정 열을 전처리하고 soynlp를 사용해 키워드 추출 결과를 저장.

    Parameters:
    - df: pd.DataFrame - 입력 데이터프레임
    - review_col: str - 리뷰 내용을 포함하는 열 이름
    - rate_col: str - 평점 열의 이름 (rate_merge=True인 경우 필요)
    - stopwords: list[str] - 필터링할 단어 목록
    - top_k: int - 추출할 상위 키워드 개수
    - full_merge: bool - True일 경우 모든 리뷰를 합쳐서 처리
    - rate_merge: bool - True일 경우 평점별로 리뷰를 합쳐서 처리

    Returns:
    - pd.DataFrame - 키워드 열이 추가된 데이터프레임
    """

    def extract_keywords_soynlp(texts):
        """soynlp WordExtractor로 키워드 추출"""
        combined_text = " ".join(texts)

        # 불용어 제거
        if stopwords:
            for word in stopwords:
                combined_text = combined_text.replace(word, '')

        # WordExtractor를 사용하여 키워드 추출
        word_extractor = WordExtractor()
        word_extractor.train([combined_text])
        words = word_extractor.extract()

        # 중요도 기준으로 상위 키워드 추출
        sorted_words = sorted(words.items(), key=lambda x: x[1].cohesion_forward, reverse=True)
        return [word for word, score in sorted_words[:top_k]]

    # 모든 리뷰를 합쳐서 처리
    if full_merge:
        merged_reviews = df[review_col].dropna().tolist()
        keywords = extract_keywords_soynlp(merged_reviews)
        return pd.DataFrame({'merge_type': ['full'], 'keywords': [keywords]})

    # 평점별로 리뷰를 합쳐 처리
    if rate_merge:
        if rate_col is None:
            raise ValueError("rate_merge=True 인 경우, rate_col을 지정해야 합니다.")

        rate_keywords = []
        for rate, group in df.groupby(rate_col):
            merged_reviews = group[review_col].dropna().tolist()
            keywords = extract_keywords_soynlp(merged_reviews)
            rate_keywords.append({'rate': rate, 'keywords': keywords})

        return pd.DataFrame(rate_keywords)

    # 기본: 각 리뷰별 처리
    soynlp_results = []
    for idx, row in df.iterrows():
        review_content = row[review_col]
        if pd.isna(review_content):
            soynlp_results.append(None)
            continue

        # 필터링 및 문장 리스트 생성
        if stopwords:
            for word in stopwords:
                review_content = review_content.replace(word, '')
        texts = review_content.split('. ')

        # 키워드 추출
        keywords = extract_keywords_soynlp(texts)
        soynlp_results.append(keywords)

    # 새로운 열 추가
    df['soynlp_keywords'] = soynlp_results
    return df

In [26]:
full_merge_df_WE = df_sumWordSentence_soynlp_WE(df, review_col='preprocessed_review', top_k=10, full_merge=True)

training was done. used memory 0.072 Gb0.072 Gb
all cohesion probabilities was computed. # words = 2525
all branching entropies was computed # words = 4729
all accessor variety was computed # words = 4729


In [27]:
full_merge_df_WE

,merge_type,keywords
0,full,"[넉넉, 꼼꼼, 퀄리, 택배, 쇼핑, 판매, 쫜득, 낱개, 쫄깃, so]"


### LRNounExtractor_v2

In [30]:
from soynlp.noun import LRNounExtractor_v2
import pandas as pd

def df_sumWordSentence_soynlp_LR(df, review_col, rate_col=None, stopwords=None, top_k=5, full_merge=False, rate_merge=False):
    """
    데이터프레임에서 특정 열을 전처리하고 soynlp를 사용해 키워드 추출 결과를 저장.

    Parameters:
    - df: pd.DataFrame - 입력 데이터프레임
    - review_col: str - 리뷰 내용을 포함하는 열 이름
    - rate_col: str - 평점 열의 이름 (rate_merge=True인 경우 필요)
    - stopwords: list[str] - 필터링할 단어 목록
    - top_k: int - 추출할 상위 키워드 개수
    - full_merge: bool - True일 경우 모든 리뷰를 합쳐서 처리
    - rate_merge: bool - True일 경우 평점별로 리뷰를 합쳐서 처리

    Returns:
    - pd.DataFrame - 키워드 열이 추가된 데이터프레임
    """

    def extract_keywords_soynlp(texts):
        """soynlp LRNounExtractor_v2로 키워드 추출"""
        combined_text = " ".join(texts)

        # 불용어 제거
        if stopwords:
            for word in stopwords:
                combined_text = combined_text.replace(word, '')

        # LRNounExtractor_v2를 사용하여 키워드 추출
        noun_extractor = LRNounExtractor_v2(verbose=False)
        nouns = noun_extractor.train_extract([combined_text])

        # 중요도 기준으로 상위 키워드 추출
        sorted_nouns = sorted(nouns.items(), key=lambda x: x[1].frequency, reverse=True)
        return [word for word, score in sorted_nouns[:top_k]]

    # 모든 리뷰를 합쳐서 처리
    if full_merge:
        merged_reviews = df[review_col].dropna().tolist()
        keywords = extract_keywords_soynlp(merged_reviews)
        return pd.DataFrame({'merge_type': ['full'], 'keywords': [keywords]})

    # 평점별로 리뷰를 합쳐 처리
    if rate_merge:
        if rate_col is None:
            raise ValueError("rate_merge=True 인 경우, rate_col을 지정해야 합니다.")

        rate_keywords = []
        for rate, group in df.groupby(rate_col):
            merged_reviews = group[review_col].dropna().tolist()
            keywords = extract_keywords_soynlp(merged_reviews)
            rate_keywords.append({'rate': rate, 'keywords': keywords})

        return pd.DataFrame(rate_keywords)

    # 기본: 각 리뷰별 처리
    soynlp_results = []
    for idx, row in df.iterrows():
        review_content = row[review_col]
        if pd.isna(review_content):
            soynlp_results.append(None)
            continue

        # 필터링 및 문장 리스트 생성
        if stopwords:
            for word in stopwords:
                review_content = review_content.replace(word, '')
        texts = review_content.split('. ')

        # 키워드 추출
        keywords = extract_keywords_soynlp(texts)
        soynlp_results.append(keywords)

    # 새로운 열 추가
    df['soynlp_keywords'] = soynlp_results
    return df

In [31]:
full_merge_df_LR = df_sumWordSentence_soynlp_LR(df, review_col='preprocessed_review', top_k=10, full_merge=True)

In [32]:
full_merge_df_LR

,merge_type,keywords
0,full,"[맛, 오란다, 딱딱, 말랑말랑, 좋아, 잘, 구매, 맛있어요, 또, 주문]"


In [40]:
full_merge_df_LR['keywords'][0][1]

'오란다'

In [50]:
# 2. 평점별로 리뷰를 묶어서 키워드 추출
rate_merge_df = df_sumWordSentence_soynlp_LR(df, review_col='preprocessed_review', rate_col='평점', rate_merge=True)

In [51]:
rate_merge_df

,rate,keywords
0,1,"[박스, 맛]"
1,2,"[맛, 물엿, 과자, 조청, 느낌]"
2,3,"[맛, 생각, 기름냄새, 딱딱, 박스]"
3,4,"[맛, 오란다, 딱딱, 많이, 생각]"
4,5,"[맛, 오란다, 딱딱, 말랑말랑, 좋아]"


In [52]:
# 3. 각 리뷰별로 처리
individual_df = df_sumWordSentence_soynlp_LR(df, review_col='preprocessed_review', top_k=5)

In [53]:
individual_df.head()

,이름,작성일자,평점,리뷰 내용,만족도,preprocessed_review,soynlp_keywords
0,솔직한깡지,2024.11.16,5,지난번 아이 생일때 선생님들 드리려고 샀다가몇개 먹어보고 맛있어서 재구매했어요 장바...,맛있어요,지난번 아이 생일때 선생님들 드리려고 샀다가몇개 먹어보고 맛있어서 재구매했어요 장바...,"[구매, 구성, 손, 맛, 바]"
1,김오성,2024.11.11,5,우주오란다 플래인 강정 주문햇습니다 이거 반틈은 내가 먹고 반틈은 울아부지 드릴껍니...,맛있어요,우주오란다 플래인 강정 주문햇습니다 이거 반틈은 내가 먹고 반틈은 울아부지 드릴껍니...,"[거의, 내]"
2,김오성,2024.09.26,5,ㅈㅣ는 84세이신 울아부지랑 둘이서 살고 있습니다 울아부지 옛날에는 장승같이 커다란...,맛있어요,ㅈㅣ는 세이신 울아부지랑 둘이서 살고 있습니다 울아부지 옛날에는 장승같이 커다란 모...,"[울아부지, 맛, 구매, 거의, 내]"
3,이*진,2024.09.22,4,《 우주오란다 》 * 용량 : 30g × 12개 * 소비기한 : 2025.04...,맛있어요,우주오란다 용량 g 개 소비기한 까지 구매 이유 어릴 때 맛나게 먹던...,"[맛, 오란다, 딱딱, 식감, 옛날]"
4,36참나리,2024.09.22,5,"클래식한 간식 중 최고봉, 오란다.달달한 간식이 생각나 주문했다.옛날 문방구에서 사...",맛있어요,클래식한 간식 중 최고봉 오란다달달한 간식이 생각나 주문했다옛날 문방구에서 사 먹던...,"[맛, 홀란드, 상인, 출신, 생각]"


## summary

In [43]:
selected_word = full_merge_df_LR['keywords'][0][1]  # 사용자가 선택한 단어

# 4. 리뷰 필터링
def filter_reviews_by_word(df, word):
    filtered_df = df[df['preprocessed_review'].str.contains(word, na=False)]
    return filtered_df

filtered_df = filter_reviews_by_word(df, selected_word)
len(filtered_df)

405

In [44]:
combined_reviews = " ".join(filtered_df['preprocessed_review'])
if isinstance(combined_reviews, list):
    combined_reviews = " ".join(combined_reviews)

In [45]:
combined_reviews

'지난번 아이 생일때 선생님들 드리려고 샀다가몇개 먹어보고 맛있어서 재구매했어요 장바구니에 넣어두고 종종보면 할인행사도 많이하더라구요 할인 전혀 없을때는 꼭 장바구니에 넣어두고 할인할때 구매하세요 ㅎㅎ맛맛은뭐 누구나 다아는 그런 오란다에요과자가 씹히면서 달달함은 덩달아 따라오는 기분좋은 맛입니다그렇다고 또 엄청나게 진짜 달다 정도는 아니에요적당한 단맛이라 저는 더 좋더라구요 너무 달면 한개다 먹기도전에 질리기도 하거든요커피랑 먹기에도 좋고 아이들은 음료나 우유랑 줘도 맛있다고 잘먹어요부드러움 정도혹시 전주에서 오란다를 먹어보신 분이라면 아실만한 그런 아주아주 부드러운 오란다에요옛날 옛적 먹던 그런 딱딱한 입천장 다까지는 오란다가 아니랍니다너무 부드러워서 저희막내는 심지어오란다를 손에 쥐어주면 동그라미 알알이 하나씩 손으로 떼어 먹어요 ㅋㅋㅋㅋㅋㅋ손은 좀 찐득해지지만 어찌나 귀여운지요두돌 조금지난 우리 막둥이도 맛나게 잘먹으니 아가들있는 집이라면딱딱하지는 않을까 걱정하진 않으셔도 되요갯수와 구성저는 이번에 개 구성으로 구매했는데요첫날에 바로 후회했어요개로 살껄 ㅜㅜ저희집 아가들이 셋인데 다들 하나씩 들고 먹으니 금방 없어지더라구요제가 저혼자만 먹을꺼 생각하고 잘못구매했다 싶었네요개들입 구성으로 구매하면 박스자체에 손잡이도 있어서혹시 선물용도 이시면 개 구성이 개당가격도 더 저렴하니 참고하세요박스를 열면 오란다는 낱개포장 되어있고한번먹기 딱좋은 양이에요외출할때 가지고 다니기도 좋고 아이들 손에 쥐어주기도 좋아서 저는 너무 만족합니다끝으로 맛있는 간식으로도 좋고 선물하기도 좋은 오란다로 추천해봅니다 ㅎㅎ다먹고 또 구매하러 올께욥 이상 솔직후기 였습니다 꾸준히 쿠팡을 사용하며 리뷰가 도움되었다는 알림에 왠지 모를 뿌듯함이 샘솟는 삼남매의 엄마랍니다오늘도 제 후기가 도움되었길 바라며  모두모두 즐거운 쇼핑하세요 우주오란다 플래인 강정 주문햇습니다 이거 반틈은 내가 먹고 반틈은 울아부지 드릴껍니더울아부지 잘드시는 우주오란다 플래인 강정 참 좋습니다 옛날에 드시던 딴딴한거

In [47]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# 모델 및 토크나이저 로드
model_name = "gogamza/kobart-summarization"  # 최신 KoBART 요약 모델
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# 텍스트 토크나이징
inputs = tokenizer([combined_reviews], max_length=1024, return_tensors="pt", truncation=True)

# 요약 생성
summary_ids = model.generate(inputs["input_ids"], max_length=512, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)

# 결과 디코딩 및 출력
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("요약 결과:")
print(summary)


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


요약 결과:
아이 생일 때 아이 생일때 선생님들 드리려고 샀다가 장바구니에 넣어두고 종종보면 할인행사도 많이하더라구요 할인 전혀 없을때는 꼭 장바구니에 넣어두고 할인할때 구매하세요 ᄒᄒ맛은뭐 다아는 그런 오란다에요과자가 씹히면서 달달함은 덩달아 따라오는 기분좋은 맛입니다그렇다고 또 엄청나게 진짜 달다 정도는 아니에요적당한 단맛이라 저는 더 좋더라구요 너무 달면 한개다 먹기도전에 질리기도 하거든요커피랑 먹기에도 좋고 아이들은 음료나 우유랑 줘도 맛있다고 잘먹어요부드러움 정도혹시 전주에서 오란다를 먹어보신 분이라면 아실만한 그런 아주아주 부드러운 오란다에요옛날 옛적 먹던 그런 딱딱한 입천장 다까지는 오란다가 아니랍니다너무 부드러워서 저희막내는 심지어오란다를 손에 쥐어주면 동그라미 알알이 하나씩 손으로 손으로 떼어 먹어요 ᄏᄏᄏᄏᄏᄏ손은 좀 찐득해지지만 어찌나 귀여운지요두돌 조금 찐득해지지만 어찌나 귀여운지요두돌 조금다른 우리 막둥이도 맛나게 잘먹으니 아가들있는 집이라면딱딱하지는 않을까 걱정하진 않으셔도 되요갯수와 구성저는 이번에 개 구성으로 구매했는데요첫날에 큰 개 구성으로 구매했는데요첫날에 깐날 큰적 먹던 그런 딱딱한 입천장 다까지는 오란다가 아니랍니다너무 부드러워                      손은 좀 찐득해지지만 어찌나 귀여운지요두돌 조금다른 우리 막둥이도 맛나게 잘먹으니 아가들있는 집이라면딱딱하지는 않을까 걱정하진 않으셔도 되요갯수와 구성저는 이번에 개 구성으로 구매했는데요첫날에 바로 후회했어요 저희집 아가들이 셋인데 다들 하나씩 들고 먹으니 금방 없어지더라구요제가 저혼자 살다 구매하면 박스자체에 손잡이도 있어서혹시 선물용도 이시면 개 구성이 개당가격도 더 저렴하니 참고하세요박스를 열면 오란다는 오란다는다는 낱개포장 되어있고 큰번먹기 딱좋은 양이에요외출할때 가지고 다니기도 좋고 아이들 손에 쥐어주기도 좋아서 저는 너무 만족합니다 ᄒᄒ다먹
